#Make sure to have a directory named "cnn_inputs"

In [1]:
import cv2 as cv
import mediapipe as mp
import time
import numpy as np
from matplotlib import pyplot as plt
import math
import os
import pathlib
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image as imagekeras
from tensorflow.keras.models import load_model
import tensorflow
from PIL import Image
import sys

### Utils

In [2]:
segments = []
recording = []
segPose = []
recPose = []

str_feed = ["Look forward", "Keep your foot flat", "Point knees outward", "Keep your torso upright", "Squat Lower"]

dir_path =f'{os.path.abspath(os.getcwd())}/cnn_inputs'
model = load_model('model.h5')

### Preprocessing

In [3]:
def segmentize(recording_arr):
  segments.append(recording_arr[0]) # top_1
  segPose.append(recPose[0])
  
  prev_frame_yval = round(recording_arr[0][1], 1) # NOTE: The 2 index here is he 2nd decimal place
  change_idx = 0

  next_frame_yval = round(recording_arr[len(recording_arr)-2][1], 1)
  change_idx_bot = 0

  for i in range(len(recording_arr)):
    if round(recording_arr[i][1], 1) > prev_frame_yval: # This is where the change starts from top_1 
      change_idx = i
      break

  for j in range(len(recording_arr)):
    if round(recording_arr[j][1], 1) == next_frame_yval: # This is where the change starts to bot_1
      change_idx_bot = j
      break

  segments.append(recording_arr[math.floor((change_idx_bot + change_idx)/2)]) # mid_1
  segments.append(recording_arr[len(recording_arr)-2]) # bot_1
  segPose.append(recPose[math.floor((change_idx_bot + change_idx)/2)])
  segPose.append(recPose[len(recording_arr)-2])

  print("Segmentized!")
  return True

In [4]:
def save_imgs():
  counter = 0
  for i in segments:
    cv.imwrite(f"{os.path.abspath(os.getcwd())}/cnn_inputs/input{counter}.png", i[0])
    counter += 1

  print("Images saved!")
  recording.clear()
  segments.clear()


### Model

In [5]:
def predict_imgs():
  predictions = []
  for i in range(4):
    img = imagekeras.load_img(f"{dir_path}/input{i}.png", target_size = (224, 224, 3))
    # plt.imshow(img)
    # plt.show()

    X = imagekeras.img_to_array(img)
    X = np.expand_dims(X, axis = 0)
    images = np.vstack([X])

    val = model.predict(images)
    class_names = {0 : 'c_bot_1', 1 :'c_mid_1', 2 :'c_top_1', 3 :'c_top_2', 4 :'i_bot_1', 5 :'i_mid_1', 6 :'i_top_1', 7 :'i_top_2'}
    scores = tensorflow.nn.softmax(val[0])
    scores = scores.numpy()
    result = f"{class_names[np.argmax(scores)]} with a { (100 *       np.max(scores)).round(2) } % confidence." 
    predictions.append(f"{class_names[np.argmax(scores)]}")
    print(f"{class_names[np.argmax(scores)]}")
  files = next(os.walk(dir_path), (None, None, []))[2]
  for file in files: 
    os.remove(f"{dir_path}/{file}")
  return predictions

In [6]:
def predictImage():
  img = imagekeras.load_img("input.png", target_size = (224, 224, 3))

  X = imagekeras.img_to_array(img)
  X = np.expand_dims(X, axis = 0)
  images = np.vstack([X])

  val = model.predict(images)
  class_names = {0 : 'c_bot_1', 1 :'c_mid_1', 2 :'c_top_1', 3 :'c_top_2', 4 :'i_bot_1', 5 :'i_mid_1', 6 :'i_top_1', 7 :'i_top_2'}
  scores = tensorflow.nn.softmax(val[0])
  scores = scores.numpy()
  result = f"{class_names[np.argmax(scores)]} with a { (100 *       np.max(scores)).round(2) } % confidence." 



### Formula

In [7]:
def cal(x1, x2, x3):
  return ((x2-x1) * (x3-x1))

def distance3d(x1, y1, z1, x2, y2, z2):
  x = (x2-x1) ** 2
  y = (y2-y1) ** 2
  z = (z2-z1) ** 2
  return math.sqrt( x + y + z )

def distance2d(x1, y1, x2, y2):
    x = x1 - x2
    y = y1 - y2
    return x **2 + y ** 2
     
def angle2d(x1, y1, x2, y2, x3,y3):
  
  # Square of lengths be a2, b2, c2
  a2 = distance2d(x2, y2, x3, y3)
  b2 = distance2d(x1, y1, x3, y3)
  c2 = distance2d(x1, y1, x2, y2)

  # length of sides be a, b, c
  a = math.sqrt(a2);
  b = math.sqrt(b2);
  c = math.sqrt(c2);

  # From Cosine law
  angle = math.acos((a2 + c2 - b2) /
                     (2 * a * c));
  # Converting to degree
  angle = angle * 180 / math.pi;
  
  return angle

def slope3d(x1, y1, z1, x2, y2, z2):
  
  deltaX = ((x1 - x2) **2)
  deltaY = (y1 - y2) 
  deltaZ = ((z1 - z2) ** 2) 
  offset = math.sqrt( deltaZ + deltaX)
  
  return ( deltaY / offset)

def angle3d(x1, y1, z1, x2, y2, z2, x3, y3, z3):
  
  # Numerator
  num = (cal(x2, x1, x3) 
          + cal(y2, y1, y3) 
          + cal(z2, z1, z3))
  #Denominator
  e1 = distance3d(x2, y2, z2, x1, y1, z1)
  e2 = distance3d(x2, y2, z2, x3, y3, z3)
  
  den = e1 * e2
  angle = math.acos(num / den)

  return angle * 180 / math.pi


def lmPosition(pose, ctr):
  x = pose.landmark[ctr].x
  y = pose.landmark[ctr].y
  z = pose.landmark[ctr].z
  return [x,y,z]


### Posture Check

In [8]:
#Remember, the functions should return true if the postures are correct

def HeadAli(pose):
  eyeOuter = lmPosition(pose, 3)
  ear = lmPosition(pose, 7)
  slope = slope3d(ear[0], ear[1], ear[2], 
                  eyeOuter[0], eyeOuter[1], eyeOuter[2])
  print(f"Head - {slope}")
  if(slope > 0.03):
    return True
  else : 
    return False
  

def FlatFoot(pose):
  leftHeel = lmPosition(pose, 30)
  leftFootidx = lmPosition(pose, 32)
  rightHeel = lmPosition(pose, 29)
  rightFootidx = lmPosition(pose, 31)
  

  
  # if( leftHeel[0] > leftFootidx[0]):
  l_slope = 100 * slope3d(leftFootidx[0], leftFootidx[1], leftFootidx[2], 
              leftHeel[0], leftHeel[1], leftHeel[2])
  # else:
  # l_slope = 100 * slope3d(leftHeel[0], leftHeel[1], leftHeel[2], 
  #               leftFootidx[0], leftFootidx[1], leftFootidx[2])
    
  print(f"l_slope - {l_slope}")
  if(l_slope < 35 ):                      #checks if right feet is flat
    return True                      #if flat, return true
  else :
    return False

def TorsoAngle(top1, pose):
  top_leftShoulder = lmPosition(top1, 12)
  top_leftHip = lmPosition(top1, 24)
  leftShoulder = lmPosition(pose, 12)
  leftHip = lmPosition(pose, 24)
  # leftKnee = lmPosition(pose, 26)
  Tslope = 100 * slope3d(top_leftHip[0], top_leftHip[1], top_leftHip[2], 
                top_leftShoulder[0], top_leftShoulder[1], top_leftShoulder[2])
  
  slope = 100 * slope3d(leftHip[0], leftHip[1], leftHip[2], 
                leftShoulder[0], leftShoulder[1], leftShoulder[2])
  
  # angle = angle3d(leftKnee[0], leftKnee[1], leftKnee[2],
  #                 leftHip[0], leftHip[1], leftHip[2],
  #                 leftShoulder[0], leftShoulder[1], leftShoulder[2])
  
  # if(i == 1 or i==2):
  #   slope = 1000 * slope3d(leftShoulder[0], leftShoulder[1], leftShoulder[2], 
  #                 leftHip[0], leftHip[1], leftHip[2])
  #   if(slope > top):
  #     return true
  print(f"top1 - {Tslope}\n mid/bot - {slope}")
  if (Tslope > slope) and (slope > 90) :
    return True
  else : 
    return False

def depth(pose):
  leftHip = lmPosition(pose, 24)
  leftKnee = lmPosition(pose, 26)
  leftAnkle = lmPosition(pose, 28)
  
  angle = angle2d(leftAnkle[0], leftAnkle[1], 
                  leftKnee[0], leftKnee[1],
                  leftHip[0], leftHip[1])
  
#   angle = angle_triangle2(leftHip[0], leftHip[1], leftHip[2], 
#                   leftKnee[0], leftKnee[1], leftKnee[2],
#                   leftAnkle[0], leftAnkle[1], leftAnkle[2])


#   if(i == 1 or i==2):
#   leftHip = lmPosition(pose, 23)
#   leftKnee = lmPosition(pose, 25)
#   leftAnkle = lmPosition(pose, 27)
  
#   angle = angle3d(leftKnee[0], leftKnee[1], leftKnee[2], 
#                   leftAnkle[0], leftAnkle[1], leftAnkle[2],
#                   leftHip[0], leftHip[1], leftHip[2])
  print(f"Depth - {angle}")
  if(angle < 75) :
    return True
  else : 
    return False


def kneeCaveIn(pose):
  rightKnee = lmPosition(pose, 25)
  leftKnee = lmPosition(pose, 26)
  rightFootidx = lmPosition(pose, 31)
  leftFootidx = lmPosition(pose, 32)

  disKnee = distance3d(rightKnee[0], rightKnee[1], rightKnee[2], leftKnee[0], leftKnee[1], leftKnee[2])

  disFI = distance3d(rightFootidx[0], rightFootidx[1], rightFootidx[2],leftFootidx[0], leftFootidx[1], leftFootidx[2])
  
  print(f"disKnee - {disKnee}\n disFI - {disFI}")
  if(disKnee > (disFI/2) ) :
    return True
  else : 
    return False


In [9]:
def feedback(poses): 
  pred = predict_imgs()
  Head = True
  Foot = True
  Knee = True
  Torso = True
  Depth = True
  seg = ["c_top_1", "c_mid_1", "c_bot_1", "c_top_2"]
  for i in range (4):
    if(pred[i] != seg[i]):
      Head = Head and HeadAli(poses[i])
      Foot = Foot and FlatFoot(poses[i])
      Knee = Knee and kneeCaveIn(poses[i])
      if(i == 1 or i == 2):
        Torso =  Torso and TorsoAngle(poses[0], poses[i])
        if(i == 2):
          Depth = Depth and depth(poses[i])
    print("")
  return [Head, Foot, Knee, Torso, Depth]

# Main 

In [10]:
# pose estimation drawing utilities
mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose
pose = mpPose.Pose()

# One time camera snapshot
camera = cv.VideoCapture(0, cv.CAP_DSHOW)
_, cam_frame = camera.read()
camera.release()
cv.destroyAllWindows()
cam_shape = cam_frame.shape

# Recording ulitilites
prev_img = np.zeros(cam_shape)

# timer utilities
initial_time = time.time()
initial_time2 = time.time()
initial_ankle_pos = 0.01
isTimed = False
isRecording = False
camera = cv.VideoCapture(0, cv.CAP_DSHOW)
tmp_lmval =  0.1

monitor_val = 0.01
isTracking = False
rec_this = 0
feed = ""
predictImage()

while True:
  
  ret, frame = camera.read()

  imgRGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
  results = pose.process(imgRGB)
    
  if results.pose_landmarks:
    mpDraw.draw_landmarks(frame, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
    
    if time.time() >= initial_time2 :
      res = [True, True, True, True, True]
      squatPrompt = "Get Ready"
      
      for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = frame.shape

        if id == 11: 
          tmp_lmval = round(lm.x, 1)

        if (not isTimed): # Set timer for 2 seconds

          initial_time = time.time()
          if id == 11: 
            initial_ankle_pos = round(lm.x, 1)
        else: 
          if time.time() >= initial_time2 + 1 :
            if time.time() >= initial_time + 1.5:
              isRecording = True
              squatPrompt = "Please Squat Now"


        if tmp_lmval == initial_ankle_pos: 
          isTimed = True
        else:
          isTimed = False

        if isRecording:
          if id == 11:
            recording.append((frame, round(lm.y, 1)))
            recPose.append(results.pose_landmarks)

            #monitor_val = round(lm.x, 1)
            rec_this = round(lm.y, 1)

            if monitor_val > rec_this:
              isRecording = False
              isTracking = segmentize(recording) 
              monitor_val = 0.01
              isTimed = False
              initial_time = time.time()

            monitor_val = round(lm.y, 1)

        if isTracking:
          if id == 11:
            if recording[0][1] == round(lm.y, 1):
              isTracking = False
              recording.append((frame, round(lm.y, 1)))
              recPose.append(results.pose_landmarks)
              segments.append(recording[len(recording)-1])
              segPose.append(recPose[len(recording)-1])
              save_imgs()
              isRecording = False
              isTimed = False
              initial_time = time.time() + 1000
              initial_time2 = time.time() + 5
              monitor_val = 0.01
              res = feedback(segPose)
              recPose.clear()
              segPose.clear()
              if res == [True, True, True, True, True] :
                squatPrompt = "You have great posture"
              else:
                squatPrompt = "Please fix your posture"
              
    for i in range(5):
      y = 100 + (50 * i)
      if not res[i]:
        frame = cv.putText(frame, str_feed[i], (40, y ), cv.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        
  else :
    squatPrompt = "Please make sure you are visible"
    
  image = cv.putText(frame, squatPrompt, (40, 50), cv.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
  
                
    
      
  cv.imshow("Realtime", image)
  key = cv.waitKey(1)

  if key == ord('q'):
      key = cv.waitKey()
      camera.release()
      cv.destroyAllWindows()
      break


Segmentized!
Images saved!
i_bot_1
i_bot_1
i_top_2
i_bot_1
Head - -0.06966360690677408
l_slope - 65.99501661261884
disKnee - 0.3036443453921942
 disFI - 0.19855427996894479

disKnee - 0.3036443453921942
 disFI - 0.19855427996894479
top1 - 36.35583221406364
 mid/bot - 36.35583221406364

disKnee - 0.3161900984133666
 disFI - 0.2585759470760954
Depth - 174.60891807487357

disKnee - 0.10956405994734605
 disFI - 0.2499853222518413

Segmentized!
Images saved!
i_bot_1
i_bot_1
i_bot_1
i_bot_1
Head - 0.12746009667123898
l_slope - 6.584911176787996
disKnee - 0.41720128703590487
 disFI - 0.7329703324186149

Head - 0.12746009667123898
l_slope - 6.584911176787996
disKnee - 0.41720128703590487
 disFI - 0.7329703324186149
top1 - 1092.3429636755984
 mid/bot - 1092.3429636755984

Head - 0.04332177868934209
l_slope - 57.9029450739012
disKnee - 0.13216680484662893
 disFI - 0.3139308591275104
Depth - 171.68347167284622

Head - 0.058225680257323914

Segmentized!
Images saved!
i_bot_1
i_bot_1
i_bot_1
i_bot_